*   분류 : 로지스틱회귀
*   회귀 : 선형회귀(다항회귀 변경 - poly 추가)
*   교차검증, 폴드를 따로 지정해서 적용
*   그리드서치, 랜덤서치



데이콘 문제 풀기

In [113]:
!unzip '/content/drive/MyDrive/데이콘/웹로그기반조회수예측.zip'

Archive:  /content/drive/MyDrive/데이콘/웹로그기반조회수예측.zip
replace open/sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: open/sample_submission.csv  
  inflating: open/test.csv           
  inflating: open/train.csv          


In [114]:
import pandas as pd

In [115]:
df = pd.read_csv('/content/open/train.csv')
df.head(1)

,sessionID,userID,TARGET,browser,OS,device,new,quality,duration,bounced,transaction,transaction_revenue,continent,subcontinent,country,traffic_source,traffic_medium,keyword,referral_path
0,SESSION_000000,USER_000000,17.0,Chrome,Macintosh,desktop,0,45.0,839.0,0,0.0,0.0,Americas,Northern America,United States,google,organic,Category8,NaN


In [116]:
X = df.loc[:,'browser':]
y = df['TARGET']
X.head(1)

,browser,OS,device,new,quality,duration,bounced,transaction,transaction_revenue,continent,subcontinent,country,traffic_source,traffic_medium,keyword,referral_path
0,Chrome,Macintosh,desktop,0,45.0,839.0,0,0.0,0.0,Americas,Northern America,United States,google,organic,Category8,NaN


In [117]:
test_df = pd.read_csv('/content/open/test.csv')
test_df = test_df[ X.columns]
test_df.head(1)

,browser,OS,device,new,quality,duration,bounced,transaction,transaction_revenue,continent,subcontinent,country,traffic_source,traffic_medium,keyword,referral_path
0,Chrome,Linux,desktop,1,75.0,698.0,0,0.0,0.0,Americas,Northern America,United States,(direct),(none),NaN,Category1


In [118]:
# 전처리 - 최빈값으로 전처리
temp = X.isna().mean()
for na_obj_col in temp[temp>0].index:
  X[na_obj_col] = X[na_obj_col].fillna(X[na_obj_col].mode()[0])

temp = test_df.isna().mean()
for na_obj_col in temp[temp>0].index:
  test_df[na_obj_col] = test_df[na_obj_col].fillna(test_df[na_obj_col].mode()[0])


In [119]:
# 2.전처리 - object를 수치형으로 변경
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

object_colunms = X.select_dtypes(include=['object']).columns
object_colunms

Index(['browser', 'OS', 'device', 'continent', 'subcontinent', 'country',
       'traffic_source', 'traffic_medium', 'keyword', 'referral_path'],
      dtype='object')

In [120]:
X.describe(include='object')

,browser,OS,device,continent,subcontinent,country,traffic_source,traffic_medium,keyword,referral_path
count,252289,252289,252289,252289,252289,252289,252289,252289,252289,252289
unique,70,21,3,6,23,205,168,7,623,1578
top,Chrome,Windows,desktop,Americas,Northern America,United States,google,organic,Category8,Category1
freq,174395,88119,167133,126506,111945,104201,116007,107370,241259,179794


In [121]:
for colname in object_colunms:
  try:
    np.unique(test_df[colname])
  except:
    print(colname)

In [122]:
# 학습용데이터를 기준으로 라벨인코딩을하는데 검증용데이터중에 추가되는 범주형 데이터가 있으면 라벨인코더에 추가한다
import numpy as np
for colname in object_colunms:
  le.fit(X[colname])
  X[colname] = le.transform(X[colname])

  for case_ in np.unique(test_df[colname]):
    if case_ not in le.classes_:
      le.classes_ = np.append(le.classes_,case_)

  test_df[colname]  =  le.transform(test_df[colname])


In [123]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(X)
X_scaled = ss.transform(X)
X_test_scaled = ss.transform(test_df)

In [ ]:
from sklearn.linear_model import LinearRegression,Ridge
from sklearn.model_selection import GridSearchCV
param = {
    'max_iter' :[None,10,100,200,500,1000,2000],
    'tol':[None,0.0001,0.0002,0.0003],
    'solver' : ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg','sag', 'saga', 'lbfgs']
}
gsc = GridSearchCV(Ridge(),param_grid=param ,n_jobs=-1 )
gsc.fit(X_scaled,y)
best_model = gsc.best_estimator_

In [ ]:
predict = best_model.predict(X_test_scaled)
submission_df = pd.read_csv('/content/open/sample_submission.csv')
submission_df['TARGET'] = predict
submission_df.to_csv("제출.csv",index=False)